# Interfaces, Patterns, and Modularity

Design patterns:

* Are reusable solutions to many common design problems appearing in software engineering. 

* Are often language-agnostic and thus can be expressed using many programming languages.


### zope.interface

Consider the following code that checks if a list of geometric figures collide with the Axis-Aligned Bounding Box (AABB) algorithm. A simple implementation would be as follows:


In [1]:
!python _01_zope_interface/colliders_simple.py


(Square(x=0, y=0, size=10), Rect(x=5, y=5, width=20, height=20))
(Square(x=0, y=0, size=10), Circle(x=1, y=1, radius=2))
(Rect(x=5, y=5, width=20, height=20), Square(x=15, y=20, size=5))


But what if we add another figure that is missing some important attribute?. We'll only see the error at runtime:


In [2]:
from _01_zope_interface.colliders_simple import Point, Square, find_collisions

try:
    coll_point = find_collisions([
        Square(0, 0, 10),
        Point(100, 200),
    ])
except AttributeError as e:
    print(f'ERROR: {repr(e)}')

ERROR: AttributeError("'Point' object has no attribute 'bounding_box'")


We can avoid those kind of errors declaring explicit interfaces. The zope.interface allows us to create and implement explicit interfaces, as follows:

In [3]:
!python _01_zope_interface/colliders_interfaces.py


Valid attempt:

(Square(x=0, y=0, size=10), Rect(x=5, y=5, width=20, height=20))
(Square(x=0, y=0, size=10), Circle(x=1, y=1, radius=2))
(Rect(x=5, y=5, width=20, height=20), Square(x=15, y=20, size=5))

Invalid attempt (a detailed exception will be raised):

Traceback (most recent call last):
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_01_zope_interface/colliders_interfaces.py", line 106, in <module>
    for collision in find_collisions([
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_01_zope_interface/colliders_interfaces.py", line 27, in find_collisions
    verifyObject(ICollidable, item)
  File "/home/work/.cache/pypoetry/virtualenvs/expert-python-programming-fourth-edition-em7utuy--py3.10/lib/python3.10/site-packages/zope/interface/verify.py", line 166, in verifyObject
    return _verify(iface, candidate, tentative, vtype='o')
  File "/home/work/.cache/pypoetry/vi

In the last code example, the `verifyObject` function (there's also a similar `verifyClass` function) is used to raise a detailed exception if one of the class instances does not implement the interface `ICollidable` correctly.


The `verifyClass` and `verifyObject` functions only verify the surface area of the interface and aren't able to traverse into attribute types. 

We can do a more in-depth verification with the `validateInvariants` function, which checks if the class has all attributes necessary to satisfy the `ICollidable` interface, but also verifies whether the structure of those attributes satisfies constraints defined in the interface.

The new example is as follows:

In [4]:
!python _01_zope_interface/colliders_invariants.py


Valid attempt:

(Square(x=0, y=0, size=10), Rect(x=5, y=5, width=20, height=20))
(Square(x=0, y=0, size=10), Circle(x=1, y=1, radius=2))
(Rect(x=5, y=5, width=20, height=20), Square(x=15, y=20, size=5))

Invalid attempt (a detailed exception will be raised):

Traceback (most recent call last):
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_01_zope_interface/colliders_invariants.py", line 120, in <module>
    for collision in find_collisions([
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Python-Programming-Fourth-Edition/Chapter_5/_01_zope_interface/colliders_invariants.py", line 27, in find_collisions
    ICollidable.validateInvariants(item)
  File "/home/work/.cache/pypoetry/virtualenvs/expert-python-programming-fourth-edition-em7utuy--py3.10/lib/python3.10/site-packages/zope/interface/interface.py", line 869, in validateInvariants
    invariant(obj)
  File "/home/work/Documents/Github/will-i-amv-books/Expert-Pyt

In the last code example, the `Point` class seems to implement the `ICollidable` interface on the surface, bit it does not satisfy the interface invariant: it doesn't have the `bounding_box` property as instance of the `Box` type, so an exception is raised.
